In [4]:
import pandas as pd
import geopy.distance

In [5]:
# Importation du jeu de données
df = pd.read_csv('datauber.csv')

In [6]:
# Suppression des colonnes inutiles
df.drop(['Unnamed: 0', 'key'], axis=1, inplace=True)

In [7]:
# Affichage des premières lignes du jeu de données
display(df.head())

,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,7.5,2015-05-07 19:52:06 UTC,-73.999817,40.738354,-73.999512,40.723217,1.0
1,7.7,2009-07-17 20:04:56 UTC,-73.994355,40.728225,-73.994710,40.750325,1.0
2,12.9,NaN,-74.005043,40.740770,NaN,40.772647,1.0
3,5.3,2009-06-26 08:22:21 UTC,-73.976124,NaN,-73.965316,40.803349,3.0
4,16.0,2014-08-28 17:47:00 UTC,NaN,40.744085,-73.973082,NaN,5.0


In [12]:
# Définition de la variable cible et des fonctionnalités
target = 'fare_amount'
features = [i for i in df.columns if i not in [target]]

In [14]:
# Affichage des informations sur le jeu de données
print(' The Datset consists of {} features & {} samples.'.format(df.shape[1], df.shape[0]))

 The Datset consists of 7 features & 44377 samples.


In [16]:
# Vérification des valeurs nulles
nvc = pd.DataFrame(df.isnull().sum().sort_values(), columns=['Total Null Values'])
nvc['Percentage'] = round(nvc['Total Null Values'] / df.shape[0], 3) * 100
print(nvc)

                   Total Null Values  Percentage
fare_amount                     2160         4.9
pickup_longitude                2194         4.9
dropoff_longitude               2194         4.9
passenger_count                 2218         5.0
pickup_latitude                 2247         5.1
dropoff_latitude                2258         5.1
pickup_datetime                 2283         5.1


In [18]:
# Suppression des lignes avec des valeurs nulles
df.dropna(inplace=True)

In [20]:
# Réorganisation des colonnes
df = df[(df.pickup_latitude < 90) & (df.dropoff_latitude < 90) &
        (df.pickup_latitude > -90) & (df.dropoff_latitude > -90) &
        (df.pickup_longitude < 180) & (df.dropoff_longitude < 180) &
        (df.pickup_longitude > -180) & (df.dropoff_longitude > -180)]

In [22]:
# Conversion de la colonne 'pickup_datetime' en datetime
df.pickup_datetime = pd.to_datetime(df.pickup_datetime)

In [23]:
# Extraction des informations temporelles
df['year'] = df.pickup_datetime.dt.year
df['month'] = df.pickup_datetime.dt.month
df['weekday'] = df.pickup_datetime.dt.weekday
df['hour'] = df.pickup_datetime.dt.hour

In [26]:
# Création de nouvelles fonctionnalités
df['Monthly_Quarter'] = df.month.map({1: 'Q1', 2: 'Q1', 3: 'Q1', 4: 'Q2', 5: 'Q2', 6: 'Q2', 7: 'Q3',
                                      8: 'Q3', 9: 'Q3', 10: 'Q4', 11: 'Q4', 12: 'Q4'})
df['Hourly_Segments'] = df.hour.map({0: 'H1', 1: 'H1', 2: 'H1', 3: 'H1', 4: 'H2', 5: 'H2', 6: 'H2', 7: 'H2', 8: 'H3',
                                     9: 'H3', 10: 'H3', 11: 'H3', 12: 'H4', 13: 'H4', 14: 'H4', 15: 'H4', 16: 'H5',
                                     17: 'H5', 18: 'H5', 19: 'H5', 20: 'H6', 21: 'H6', 22: 'H6', 23: 'H6'})

In [28]:
# Calcul de la distance entre les points de prise en charge et de dépose
df['Distance'] = [round(geopy.distance.distance((df.pickup_latitude[i], df.pickup_longitude[i]),
                                                 (df.dropoff_latitude[i], df.dropoff_longitude[i])).m, 2)
                  for i in df.index]

In [29]:
# Suppression des colonnes inutiles
df.drop(['pickup_datetime', 'month', 'hour'], axis=1, inplace=True)

In [15]:
# Sauvegarde du jeu de données original
original_df = df.copy(deep=True)

In [30]:
# Sauvegarde du nouveau jeu de données nettoyé
df.to_csv("nouveau_fichier_nettoye11.csv", index=False)

In [31]:
# Affichage des premières lignes du jeu de données nettoyé
df.head()

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,year,weekday,Monthly_Quarter,Hourly_Segments,Distance
0,7.5,-73.999817,40.738354,-73.999512,40.723217,1.0,2015,3,Q2,H5,1681.11
1,7.7,-73.994355,40.728225,-73.994710,40.750325,1.0,2009,4,Q3,H6,2454.36
5,4.9,-73.969019,40.755910,-73.969019,40.755910,1.0,2011,5,Q1,H1,0.00
7,2.5,0.000000,0.000000,0.000000,0.000000,1.0,2012,1,Q4,H4,0.00
8,9.7,-73.975187,40.745767,-74.002720,40.743537,1.0,2012,4,Q1,H3,2338.56


In [36]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from tqdm import tqdm

# Charger les données
data = pd.read_csv("nouveau_fichier_nettoye11.csv")

# Sélectionner les colonnes spécifiées
selected_columns = ['pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude', 'passenger_count', 'year', 'weekday', 'Monthly_Quarter', 'Hourly_Segments', 'Distance']
X = data[selected_columns]
Y = data["fare_amount"]

# Convertir les variables catégorielles en variables binaires
X = pd.get_dummies(X, columns=['Monthly_Quarter', 'Hourly_Segments'])

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Créer une barre de progression
pbar = tqdm(total=100, desc="Entraînement du modèle")

# Entraîner le modèle de régression linéaire multiple
model = LinearRegression()
model.fit(X_train, Y_train)

# Mettre à jour la barre de progression
pbar.update(100)
pbar.close()

# Faire des prédictions sur l'ensemble de test
predictions = model.predict(X_test)

# Évaluer le modèle
rmse = mean_squared_error(Y_test, predictions, squared=False)
print("RMSE:", rmse)


Entraînement du modèle: 100%|██████████| 100/100 [00:00<00:00, 498.14it/s]

RMSE: 10.286288161944656


In [39]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Charger les données
data = pd.read_csv("nouveau_fichier_nettoye11.csv")

# Sélectionner les colonnes spécifiées
selected_columns = ['pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude', 'passenger_count', 'year', 'weekday', 'Monthly_Quarter', 'Hourly_Segments', 'Distance']
X = data[selected_columns]
Y = data["fare_amount"]

# Convertir les variables catégorielles en variables binaires
X = pd.get_dummies(X, columns=['Monthly_Quarter', 'Hourly_Segments'])

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Créer et entraîner le modèle de forêt aléatoire avec moins d'estimateurs
model = RandomForestRegressor(n_estimators=50, random_state=42)  # Moins d'estimateurs pour réduire le temps d'exécution
model.fit(X_train, Y_train)

# Faire des prédictions sur l'ensemble de test
predictions = model.predict(X_test)

# Évaluer le modèle
rmse = mean_squared_error(Y_test, predictions, squared=False)
print("RMSE:", rmse)


RMSE: 5.2330585140467205
